In [ ]:
import pandas as pd
data = pd.read_csv("./BostonHousing.csv")
data.head(3)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
knn_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('scale',StandardScaler())
])

# sc = MinMaxScaler(feature_range=(0,1))

In [ ]:
x = data.iloc[:,:-1]

In [ ]:
x = knn_pipeline.fit_transform(x)
x

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

In [ ]:
x.shape

(506, 13)

In [ ]:
y = data.iloc[:,-1]
y.shape

(506,)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state = 1)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=3)
model.fit(xtrain,ytrain)

KNeighborsRegressor(n_neighbors=3)

In [ ]:
predictions = model.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score
r2_score(ytest, predictions)

0.8276227796691884

Find best number of neighbors using **gridSearchCV**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model2 = KNeighborsRegressor()

In [ ]:
param = {'weights':['uniform','distance'], "n_neighbors":[3,5,7,9,11]}

scoring = 'neg_mean_squared_error'

In [ ]:
search = GridSearchCV(model2, param, scoring = scoring, cv = 5)
search.fit(xtrain, ytrain)
print(f"Best Params: {search.best_params_}")

Best Params: {'n_neighbors': 3, 'weights': 'distance'}


In [ ]:
final_model = search.best_estimator_

In [ ]:
predictions2 = final_model.predict(xtest)

from sklearn.metrics import r2_score
r2_score(ytest, predictions2)

0.8638489106891928

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_params = [
{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
forest_reg = RandomForestRegressor()
forest_grid_search = GridSearchCV(forest_reg,forest_params,cv=5,scoring='neg_mean_squared_error',return_train_score=True)
forest_grid_search.fit(xtrain, ytrain)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [ ]:
-forest_grid_search.best_score_
final_forest_reg_model = forest_grid_search.best_estimator_

In [ ]:
final_predictions = final_forest_reg_model.predict(xtest)


In [ ]:
from sklearn.metrics import r2_score
r2_score(ytest, final_predictions)

0.895847817985527

**Decision Tree**